In [1]:
!pip install gym_super_mario_bros nes_py


In [1]:
import gym_super_mario_bros
#import the joypad wrapper
from nes_py.wrappers import JoypadSpace
#import the simplified controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [2]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

In [3]:
# Setup Game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT) #Simplifies the action space

In [4]:
env.action_space

Discrete(7)

In [5]:
env.observation_space.shape

(240, 256, 3)

In [11]:
# random action mario game
done = True
for step in range(100000):
    if done:
        env.reset()
    state,reward,done,info = env.step(env.action_space.sample())
    env.render()
env.close()

KeyboardInterrupt: 

In [6]:
env.close()

In [19]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113

In [19]:
!pip install --user stable-baselines3[extra]

  Using cached AutoROM-0.4.2-py3-none-any.whl (16 kB)
  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
  Using cached ale_py-0.7.5-cp39-cp39-win_amd64.whl (935 kB)
  Using cached importlib_metadata-4.12.0-py3-none-any.whl (21 kB)
  Using cached AutoROM.accept_rom_license-0.4.2-py3-none-any.whl
  Using cached google_auth-2.9.0-py2.py3-none-any.whl (167 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached absl_py-1.1.0-py3-none-any.whl (123 kB)
  Using cached grpcio-1.47.0-cp39-cp39-win_amd64.whl (3.6 MB)
  Using cached Markdown-3.3.7-py3-none-any.whl (97 kB)
  Using cached cachetools-5.2.0-py3-none-any.whl (9.3 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


# Preprocess Environment

In [6]:
# Import frame stacker wrapper and grayscaling wrapper
from gym.wrappers import  GrayScaleObservation
# Import Vectorization wrapper
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import matplotlib
from matplotlib import pyplot as plt

In [7]:
# Setup Game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
#Simplifies the action space
env = JoypadSpace(env, SIMPLE_MOVEMENT)
#1. GrayScale
env = GrayScaleObservation(env,keep_dim=True)
#2. Wrap in dummy environment
env = DummyVecEnv([lambda: env])
#3. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [8]:
state = env.reset()

In [9]:
state.shape

(1, 240, 256, 4)

# 2.Train RL Model

In [10]:
import os
#import ppo for algos
from stable_baselines3 import PPO
#import base callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [11]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [12]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [13]:
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)


In [14]:
model = PPO('CnnPolicy',env,verbose=1,tensorboard_log=LOG_DIR,learning_rate=0.000001,n_steps=512)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [15]:
import torch
torch.cuda.is_available()


False

In [ ]:
model.learn(total_timesteps=1000000, callback=callback)

Logging to ./logs/PPO_1
----------------------------
| time/              |     |
|    fps             | 41  |
|    iterations      | 1   |
|    time_elapsed    | 12  |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 9             |
|    iterations           | 2             |
|    time_elapsed         | 106           |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 2.8905692e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00378      |
|    learning_rate        | 1e-06         |
|    loss                 | 115           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000272     |
|    value_loss           | 330           |
-------------------------

-------------------------------------------
| time/                   |               |
|    fps                  | 5             |
|    iterations           | 13            |
|    time_elapsed         | 1238          |
|    total_timesteps      | 6656          |
| train/                  |               |
|    approx_kl            | 2.4326262e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.0111        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.062         |
|    n_updates            | 120           |
|    policy_gradient_loss | -0.000478     |
|    value_loss           | 0.224         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 5             |
|    iterations           | 14            |
|    time_elapsed         | 1364